In [1]:
# Imports para crear script

import pandas as pd
from datetime import datetime
import json

from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
# Declare file locations
input_excel = 'input/InventarioLast.xlsx'
input_email = "input/email_inserts.sql"
input_assets_ddl = "input/assets_ddl.sql"
output_sql_inserts = 'output/AssetsInserts.sql'

In [3]:
df = pd.read_excel(input_excel, skiprows=6, usecols='B:Z')
# Inicialización
lista = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]  # 0: Equipos, 1: Software, 2: Accesorios

# Columnas por tipo
lista_equipos = ['PLACA', 'SUBTIPO', 'MARCA', 'MODELO', 'PANTALLA', 'PROCESADOR',
                'GENERACIÓN', 'MEMORIA', 'DISCO DURO', 'SERIAL FISICO', 'OBSERVACIONES',
                'SERIAL BOARD', 'ESTADO', 'UBICACIÓN', 'ASIGNADO A', 'ACTA DE ENTREGA',
                'Canon', 'SISTEMA OPERATIVO', 'MOVIMIENTO', 'Comprado a', 'Factura #',
                'Nuevo / Usado', 'Garantia', 'Fecha Compra', 'Fecha Fin']

lista_software = ['SUBTIPO', 'MARCA', 'MODELO', 'ESTADO', 'UBICACIÓN', 'ASIGNADO A',
                  'ACTA DE ENTREGA', 'Canon', 'MOVIMIENTO']

lista_accesorios = ['PLACA', 'SUBTIPO', 'MARCA', 'MODELO', 'SERIAL FISICO', 'OBSERVACIONES',
                    'ESTADO', 'UBICACIÓN', 'ASIGNADO A', 'ACTA DE ENTREGA', 'Canon',
                    'MOVIMIENTO', 'Comprado a', 'Factura #', 'Nuevo / Usado', 'Garantia',
                    'Fecha Compra']

# Variable que rastrea en qué sección estamos
seccion_actual = 0  # 0 = equipos, 1 = software, 2 = accesorios

# Recorremos el DataFrame fila por fila
for idx, fila in df.iterrows():
    placa = str(fila['PLACA']).strip()

    # Cambiar de sección si encontramos un encabezado
    if placa.lower() == 'software':
        seccion_actual = 1
        continue  # no agregamos esta fila
    elif placa.lower() == 'accesorios':
        seccion_actual = 2
        continue  # no agregamos esta fila
    elif placa.lower() == '':  # si la placa está vacía, probablemente fila vacía
        continue
    elif placa.lower() == 'nomenclatura':
        break

    # Definir columnas según la sección
    if seccion_actual == 0:
        columnas = lista_equipos
    elif seccion_actual == 1:
        columnas = lista_software
    else:
        columnas = lista_accesorios

    # Agregamos solo la fila actual, con las columnas correspondientes
    lista[seccion_actual] = pd.concat([lista[seccion_actual], fila[columnas].to_frame().T], ignore_index=True)

In [4]:
# General auxiliar functions
import re
import numpy as np
import unicodedata

def quitar_tildes(texto):
    if isinstance(texto, str):
        return ''.join(
            c for c in unicodedata.normalize('NFD', texto)
            if unicodedata.category(c) != 'Mn'
        )
    return texto

def normalize_dataframe(df):
    df.columns = [quitar_tildes(col) for col in df.columns]
    for col in df.select_dtypes(include=["object", "string"]):
        df[col] = df[col].map(quitar_tildes)
    return df

def to_uppercase_all(df):
    df_upper = df.copy()

    # Convertir columnas de texto a mayúsculas
    for col in df_upper.columns:
        if df_upper[col].dtype == object or df_upper[col].dtype.name == 'category':
            df_upper[col] = df_upper[col].astype(str).str.upper()

    # Convertir nombres de columnas a mayúsculas
    df_upper.columns = [col.upper() for col in df_upper.columns]

    return df_upper

def clean_spaces(df):
    df_clean = df.copy()
    for col in df_clean.columns:
        if df_clean[col].dtype == object or df_clean[col].dtype.name == 'category':
            df_clean[col] = df_clean[col].astype(str).apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
    return df_clean


def format_df(df):
  df_out = to_uppercase_all(df)
  df_out = clean_spaces(df_out)
  df_out = df_out.replace("NAN", np.nan)
  df_out = df_out.dropna(how='all')
  return df_out

def equipos_preprocessing(equipos):
    equipos["DISCO DURO"] = equipos["DISCO DURO"].replace({"250 GB":"250GB", "1 TB": "1TB"})
    equipos["GARANTIA"] = equipos["GARANTIA"].replace({"12 MESS": "12 MESES"})
    equipos["NUEVO / USADO"] = equipos["NUEVO / USADO"].replace({"USADA": "USADO", "USADI": "USADO", np.nan: "USADO"})
    equipos
    
    equipos = equipos.rename(columns={"COMPRADO A": "PROVEEDOR"})

    return equipos

def accesorios_preprocessing(accesorios):
  accesorios["NUEVO / USADO"] = accesorios["NUEVO / USADO"].replace({"NUEVA": "NUEVO", np.nan: "USADO"})
  return accesorios

def software_preprocessing(software):
  return software

In [5]:
EQUIPOS = format_df(lista[0])
#SOFTWARE = format_df(lista[1])
#ACCESORIOS = format_df(lista[2])

equipos = equipos_preprocessing(EQUIPOS)
#accesorios = accesorios_preprocessing(ACCESORIOS)
#software = software_preprocessing(SOFTWARE)

#accesorios = normalize_dataframe(accesorios.astype(object))
#software = normalize_dataframe(software.astype(object))
equipos = normalize_dataframe(equipos.astype(object))


/tmp/ipykernel_2265/3340632672.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_out = df_out.replace("NAN", np.nan)


## Creación de script de inserts para Equipos

### Conexión a la base de datos de Procytec

In [6]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Crear conexión con SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

clients: pd.DataFrame = pd.read_sql("SELECT vc.* FROM vista_customers vc WHERE vc.codigo_cliente != 0;", engine)

### Funciones y variables de apoyo para inserts

In [7]:
def convert_to_string(val) -> str:
    if pd.isna(val) or str(val).strip() == "":
        return "NULL"
    return "'" + str(val).replace("'", "''") + "'"

def format_date(fecha_raw: str) -> str:
  try:
    fecha_dt = pd.to_datetime(fecha_raw, dayfirst=True, errors='coerce')
    fecha_val = f"'{fecha_dt.date().isoformat()}'" if not pd.isna(fecha_dt) else "NULL"
  except:
    fecha_val = "NULL"

  return fecha_val

def obtener_tipo_servicio(acta_id: str) -> str:
  if pd.isna(acta_id) or str(acta_id).strip() == "":
    return "NULL"
  acta_id = str(acta_id).strip().upper()
  if acta_id.startswith("A"):
    return "'ARRENDAMIENTO'"
  elif acta_id.startswith("Y"):
    return "'IMPRESION'"
  elif acta_id.startswith("P"):
    return "'PRESTAMO'"
  else:
    return "'OTRO'"

def extraer_codigo_cliente(acta: str) -> str:
    if pd.isna(acta) or str(acta).strip() == "":
        return "NULL"
    acta = str(acta).strip().upper()
    match = re.search(r'^[A-Z]([0-9]+)-', acta)
    if match:
        return match.group(1)
    return "NULL"

def safe_int(value, default=None):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

# Definición de constantes "ARRENDADO", "PRESTADO", "STOCK-PENDIENTE"

SUBESTADOS_ACTIVOS = ["ASIGNADO", "PEND. DEV. CLIENTE", "REPARACION", "STOCK", "REVISION", "PEND. X REPUESTO", "ACONDICIONAMIENTO", "GARANTIA"]
SUBESTADOS_OBSOLETOS = ["VENTA", "DONACION", "DAÑO"]
SUBESTADOS_RETIRADOS = ["BASURA ELECTRONICA", "VENDIDO", "ROBADO", "PERDIDO", "EXTRAER REPUESTOS"]

ESTADO_SUBESTADO = {
  **dict.fromkeys(SUBESTADOS_ACTIVOS, "ACTIVO"),
  **dict.fromkeys(SUBESTADOS_OBSOLETOS, "OBSOLETO"),
  **dict.fromkeys(SUBESTADOS_RETIRADOS, "RETIRADO")
}

ATRIBUTOS_DE_UN_ACTIVO = [
  "ACTA DE ENTREGA",
  "SUBTIPO",
  "MARCA",
  "MODELO",
  "PROVEEDOR",
  "ESTADO",
  "UBICACION",
  "ASIGNADO A",
  "FACTURA #",
  "NUEVO / USADO",
  "GARANTIA",
  "FECHA COMPRA",
  "FECHA FIN",
]

COLUMNAS_CON_VALORES_VALIDOS = [
  "PANTALLA",
  "SISTEMA OPERATIVO",
  "MEMORIA",
  "DISCO DURO",
  "GENERACION",
  "PROCESADOR",
  "PROVEEDOR"
]

COLUMNAS_SIN_VALORES_VALIDOS = [
  "PLACA",
  "SERIAL FISICO",
  "SERIAL BOARD",
]

ATRIBUTOS_ADICIONALES = COLUMNAS_CON_VALORES_VALIDOS + COLUMNAS_SIN_VALORES_VALIDOS

ATRIBUTO_IDS = {nombre: idx + 1 for idx, nombre in enumerate(ATRIBUTOS_ADICIONALES)}

CLIENTES = clients[['name', 'codigo_cliente']]
CLIENTES = CLIENTES.drop_duplicates(subset=["codigo_cliente"], keep="first")

### Tabla `clients`

In [8]:
def insert_into_clients(clients: list[str]) -> str:
    # dictionary with iterrows
    values: str = ""
    for idx, row in clients.iterrows():
        values += f"    ({idx + 1}, '{row['codigo_cliente']}', '{row['name']}'),\n"

    comma_pos = values.rfind(",")
    values = values[:comma_pos] + ";" + values[comma_pos + 1 :]

    return f"INSERT INTO clients (id, client_code, name) VALUES\n{values};"

### Tabla `estado_acta`

In [9]:
def insert_into_estado_acta() -> str:
  return """INSERT INTO estado_acta (estado_acta) VALUES
    ('ACTIVA'),
    ('BLOQUEADA'),
    ('HISTORICO'),
    ('PRE-ACTIVA'),
    ('CERRADA');\n"""

### Tabla `tipo_servicio`

In [10]:
def insert_into_tipo_servicio() -> str:
  return """INSERT INTO tipo_servicio (servicio, prefijo) VALUES
    ('IMPRESION','Y'),
    ('ARRENDAMIENTO','A'),
    ('OTRO','?'),
    ('PRESTAMO','P');\n"""

### Tabla `estado_activo`

In [11]:
def insert_into_estado_activo() -> str:
  return """INSERT INTO estado_activo (estado) VALUES
    ('ACTIVO'),
    ('OBSOLETO'),
    ('RETIRADO');\n"""

### Tabla `subestado_activo`

In [12]:
def insert_into_subestado_activo() -> str:
  values: str = ""
  for subestado, estado in ESTADO_SUBESTADO.items():
    values = values + f"    ('{subestado}', '{estado}'),\n"
  else:
    comma_pos = values.rfind(",")
    values = values[:comma_pos] + ";" + values[comma_pos+1:]

  return "INSERT INTO subestado_activo (subestado, estado_padre) VALUES\n" + values


### Tabla `atributo`

In [13]:
def insert_into_atributo(equipos: pd.DataFrame) -> str:
  valores_validos = {}
  for columna in COLUMNAS_CON_VALORES_VALIDOS:
    valores_unicos = list(equipos[columna].dropna().unique())
    valores_validos[columna] = valores_unicos

  values: str = ""
  atributo_id = 1
  for columna, valores in valores_validos.items():
    valores_json = json.dumps(valores, ensure_ascii=False)
    values += f"    ({atributo_id}, '{columna}', 'String', '{valores_json}'::jsonb, true),\n"
    atributo_id += 1

  for columna in COLUMNAS_SIN_VALORES_VALIDOS:
    values += f"    ({atributo_id}, '{columna}', 'NA', NULL, false),\n"
    atributo_id += 1
  else:
    comma_pos = values.rfind(",")
    values = values[:comma_pos] + ";" + values[comma_pos+1:]

  return (
      "INSERT INTO atributo "
      "(atributo_id, nombre, tipo_dato, valores_validos, tiene_valores_validos) VALUES\n"
      + values
  )

### Tabla `tipo_activo`



In [14]:
def insert_into_tipo_activo(equipos: pd.DataFrame) -> str:
  tipos_de_equipos = list(equipos['SUBTIPO'].dropna().unique())
  values: str = ""
  tipo_id: int = 1

  for tipo in tipos_de_equipos:
    # Filtrar equipos por tipo
    equipos_tipo = equipos[equipos['SUBTIPO'] == tipo]
    
    # Agrupar por marca y obtener modelos únicos
    marcas_modelos = []
    for marca, grupo in equipos_tipo.groupby('MARCA'):
      if pd.notna(marca) and str(marca).strip():
        modelos = list(grupo['MODELO'].dropna().unique())
        modelos = [str(modelo).strip() for modelo in modelos if str(modelo).strip()]
        if modelos:  # Solo incluir marcas que tengan al menos un modelo
          marcas_modelos.append({
            "nombre": str(marca).strip(),
            "modelos": modelos
          })
    
    # Convertir a JSON
    marcas_json = json.dumps(marcas_modelos, ensure_ascii=False)
    values += f"    ('{tipo}', '{marcas_json}'::jsonb),\n"
    tipo_id += 1
  else:
    comma_pos = values.rfind(",")
    values = values[:comma_pos] + ";" + values[comma_pos+1:]

  return "INSERT INTO tipo_activo (tipo_activo, marcas_y_modelos) VALUES\n" + values

### Tabla `atributo_tipoactivo`

In [15]:
def insert_into_atributo_tipoactivo(equipos: pd.DataFrame, atributo_ids: dict[str, int]) -> str:
  atributos_validos = [a for a in ATRIBUTOS_ADICIONALES if a not in ATRIBUTOS_DE_UN_ACTIVO]

  values = ""

  for tipo, grupo in equipos.groupby("SUBTIPO"):
      for atributo in atributos_validos:
          if grupo[atributo].notna().any(): # si el atributo aparece al menos una vez para el tipo especifico
              atributo_id = atributo_ids[atributo]
              values += f"    ('{tipo}', {atributo_id}),\n"

  comma_pos = values.rfind(",")
  values = values[:comma_pos] + ";" + values[comma_pos + 1 :]

  return "INSERT INTO atributo_tipoactivo (tipo_activo, atributo_id) VALUES\n" + values

### Tabla `activo`

In [16]:
def row_to_activo(row: pd.Series, insert_index: int) -> str:
  acta_entrega_actual: str = convert_to_string(row.get("ACTA DE ENTREGA"))
  tipo: str = convert_to_string(str(row.get("SUBTIPO", "")).strip().upper())
  marca: str = convert_to_string(row.get('MARCA'))
  modelo: str = convert_to_string(row.get('MODELO'))
  proveedor: str = convert_to_string(row.get('PROVEEDOR')) # preguntar a juank - creo que este es el comprado a
  subestado: str = convert_to_string(row.get("ESTADO"))
  estado: str = convert_to_string(ESTADO_SUBESTADO.get(row.get("ESTADO")))
  condicion: str = convert_to_string(row.get('NUEVO / USADO'))
  fecha_compra: str = format_date(str(row.get("FECHA COMPRA", "")).strip())
  factura_compra: str = convert_to_string(row.get('FACTURA #'))
  garantia: str = convert_to_string(row.get('GARANTIA'))
  asignado_a: str = convert_to_string(row.get('ASIGNADO A')) # este posiblemente queda null si es el asignado que pone el cliente manualmente
  ubicacion: str = convert_to_string(row.get('UBICACION'))

  return """INSERT INTO activo (activo_id, acta_entrega_actual, tipo, marca, modelo, proveedor, estado, subestado, condicion, fecha_compra, factura_compra, garantia, asignado_a, ubicacion) VALUES (
    {}, -- activo_id
    {}, -- acta_entrega_actual
    {}, -- tipo
    {}, -- marca
    {}, -- modelo
    {}, -- proveedor
    {}, -- estado
    {}, -- subestado
    {}, -- condicion
    {}, -- fecha_compra
    {}, -- factura_compra
    {}, -- garantia
    {}, -- asignado_a
    {}); -- ubicacion""".format(
            insert_index,
            acta_entrega_actual,
            tipo,
            marca,
            modelo,
            proveedor,
            estado,
            subestado,
            condicion,
            fecha_compra,
            factura_compra,
            garantia,
            asignado_a,
            ubicacion
        )

### Tabla `atributo_activo`

In [17]:
def row_to_atributo_activo(row: pd.Series, insert_index: int, atributo_ids: dict[str, int]) -> str:
  values: str = ""
  for columna in row.index:
    valor_columna = row[columna]
    if pd.notna(valor_columna) and str(valor_columna).strip():
      atributo_id = atributo_ids.get(columna) # busca la columna en la lista de atributos adicionales
      if atributo_id:
        values += f"    ({insert_index}, {atributo_id}, {convert_to_string(valor_columna)}), -- {columna} \n"
  else:
    comma_pos = values.rfind(",")
    values = values[:comma_pos] + ";" + values[comma_pos+1:]

  return "\nINSERT INTO atributo_activo (activo_id, atributo_id, valor) VALUES\n" + values if values != "" else "-- no posee atributos adicionales"

### Tabla `observacion`

In [18]:
def row_to_observacion(row: pd.Series, activo_id: int, observacion_id: int) -> str:
  observacion: str = convert_to_string(row.get("OBSERVACIONES"))
  return "INSERT INTO observacion (observacion_id, activo_id, contenido, fecha) VALUES ({},{},{},{});".format(
    observacion_id,
    activo_id,
    observacion,
    format_date(datetime.today().date())
  ) if observacion != "NULL" else "-- no posee observaciones"

### Tabla `movimiento`

In [19]:
def row_to_movimiento(row: pd.Series, activo_id: int, movimiento_id: int) -> str:
  movimiento: str = convert_to_string(row.get("MOVIMIENTO"))
  return "INSERT INTO movimiento (movimiento_id, activo_id, contenido, fecha) VALUES ({},{},{},{});".format(
    movimiento_id,
    activo_id,
    movimiento,
    format_date(datetime.today().date())
  ) if movimiento != "NULL" else "-- no posee movimientos"

### Tabla acta_entrega

In [20]:
def row_to_acta_entrega(row: pd.Series, clients: pd.DataFrame, actas_procesadas: set) -> str:
  actaentrega_id_raw = row.get("ACTA DE ENTREGA")
  actaentrega_id: str = convert_to_string(actaentrega_id_raw)
  
  if actaentrega_id == "NULL":
    return "-- no posee acta de entrega"
  
  # Verificar si esta acta ya fue procesada
  acta_key = str(actaentrega_id_raw).strip().upper() if pd.notna(actaentrega_id_raw) else ""
  if acta_key in actas_procesadas:
    return f"-- acta {actaentrega_id} ya existe, omitiendo insert"
  
  # Marcar esta acta como procesada
  actas_procesadas.add(acta_key)
  
  fecha_inicio: str = 'NULL' # no hay fecha de inicio en el excel
  fecha_fin: str = format_date(row.get("FECHA FIN"))
  tipo_servicio: str = obtener_tipo_servicio(row.get("ACTA DE ENTREGA"))
  estado: str = convert_to_string("ACTIVA") # por defecto todos son activos
  cliente_codigo = safe_int(extraer_codigo_cliente(row.get("ACTA DE ENTREGA")))

  cliente_row = clients[clients['codigo_cliente'] == cliente_codigo]
  cliente_id = cliente_row.index[0] + 1 if not cliente_row.empty else "NULL"

  return """INSERT INTO acta_entrega (actaentrega_id, fecha_inicio, fecha_fin, tipo_servicio, estado, cliente_id) VALUES (
    {}, -- actaentrega_id
    {}, -- fecha_inicio
    {}, -- fecha_fin
    {}, -- tipo_servicio
    {}, -- estado
    {}); -- cliente_id""".format(
            actaentrega_id,
            fecha_inicio,
            fecha_fin,
            tipo_servicio,
            estado,
            cliente_id
        )

### Tabla etiqueta

In [21]:
def row_to_etiqueta(row: pd.Series, etiqueta_id: int) -> str:
  canon: str = convert_to_string(row.get("CANON"))
  nombre: str = convert_to_string(f"ETIQUETA-{etiqueta_id:05d}")
  acta_asociada: str = convert_to_string(row.get("ACTA DE ENTREGA"))
  return "INSERT INTO etiqueta (etiqueta_id, canon, is_enabled, nombre, acta_asociada) VALUES ({},{},{},{},{});".format(
    etiqueta_id,
    canon,
    'true',
    nombre,
    acta_asociada
  ) if acta_asociada != "NULL" else "-- no posee etiqueta"

### Tabla activo_actaentrega

In [22]:
def row_to_activo_actaentrega(row: pd.Series, activo_id: int, etiqueta_id: int) -> str:
  actaentrega_id: str = convert_to_string(row.get("ACTA DE ENTREGA"))
  asignado_a: str = "''"
  descripcion: str = "''"

  etiqueta_asociada: str = etiqueta_id if actaentrega_id != "NULL" else "NULL"

  return "INSERT INTO activo_actaentrega (asignado_a, descripcion, actaentrega_id, activo_id, etiqueta_asociada) VALUES ({},{},{},{},{});".format(
    asignado_a,
    descripcion,
    actaentrega_id,
    activo_id,
    etiqueta_asociada
  ) if actaentrega_id != "NULL" else "-- no posee acta de entrega"

### Script para reiniciar los indices de las tablas

In [23]:
def reset_sequences() -> str:

    return """-- Ajustar secuencia de clients.id
SELECT setval(pg_get_serial_sequence('clients', 'id'), COALESCE((SELECT MAX(id) FROM clients), 0) + 1, false);

-- Ajustar secuencia de atributo.atributo_id
SELECT setval(pg_get_serial_sequence('atributo', 'atributo_id'), COALESCE((SELECT MAX(atributo_id) FROM atributo), 0) + 1, false);

-- Ajustar secuencia de etiqueta.etiqueta_id
SELECT setval(pg_get_serial_sequence('etiqueta', 'etiqueta_id'), COALESCE((SELECT MAX(etiqueta_id) FROM etiqueta), 0) + 1, false);

-- Ajustar secuencia de activo.activo_id
SELECT setval(pg_get_serial_sequence('activo', 'activo_id'), COALESCE((SELECT MAX(activo_id) FROM activo), 0) + 1, false);

-- Ajustar secuencia de movimiento.movimiento_id
SELECT setval(pg_get_serial_sequence('movimiento', 'movimiento_id'), COALESCE((SELECT MAX(movimiento_id) FROM movimiento), 0) + 1, false);

-- Ajustar secuencia de observacion.observacion_id
SELECT setval(pg_get_serial_sequence('observacion', 'observacion_id'), COALESCE((SELECT MAX(observacion_id) FROM observacion), 0) + 1, false);"""

## SQL merge

In [24]:
output_sql: list[str] = []

with open(input_email, "r", encoding="utf-8") as ddl_file:
  email_content = ddl_file.read()

output_sql.append("/* ======= EMAIL TEMPLATES ======= */\n")
output_sql.append(email_content)

# leer primero archivo ddl
with open(input_assets_ddl, "r", encoding="utf-8") as ddl_file:
  ddl_content = ddl_file.read()

output_sql.append("/* ======= DDL ======= */\n")
output_sql.append(ddl_content)

output_sql.append("/* ======= INSERTS ======= */\n")

clientes_insert = insert_into_clients(CLIENTES)
tipos_servicios_insert = insert_into_tipo_servicio()
estados_actas_insert = insert_into_estado_acta()
estados_activos_insert = insert_into_estado_activo()
subestados_activos_insert = insert_into_subestado_activo()
atributos_insert = insert_into_atributo(equipos)
tipos_activos_insert = insert_into_tipo_activo(equipos)
atributos_tipoactivo_insert = insert_into_atributo_tipoactivo(equipos, ATRIBUTO_IDS)

output_sql.append(clientes_insert)
output_sql.append(tipos_servicios_insert)
output_sql.append(estados_actas_insert)
output_sql.append(estados_activos_insert)
output_sql.append(subestados_activos_insert)
output_sql.append(atributos_insert)
output_sql.append(tipos_activos_insert)
output_sql.append(atributos_tipoactivo_insert)

# Inicializar contadores independientes
insert_index = 1
observacion_index = 1
movimiento_index = 1
etiqueta_index = 1

# Conjunto para llevar registro de actas ya procesadas
actas_procesadas = set()

# Agregar comentario explicativo
output_sql.append("\n/* ======= DATOS DE ACTIVOS ======= */")
output_sql.append("/* NOTA: Las actas duplicadas solo se insertan una vez. */")
output_sql.append("/* Cada activo siempre recibe su propia etiqueta y relación activo_actaentrega. */\n")

for _, row in equipos.iterrows():
  output_sql.append(f"\n/* ======= activo {insert_index} ======= */")

  activo_insert = row_to_activo(row, insert_index)
  atributo_activo_insert = row_to_atributo_activo(row, insert_index, ATRIBUTO_IDS)

  observacion_insert = row_to_observacion(row, insert_index, observacion_index)
  if not observacion_insert.startswith("--"):
    observacion_index += 1

  movimiento_insert = row_to_movimiento(row, insert_index, movimiento_index)
  if not movimiento_insert.startswith("--"):
    movimiento_index += 1

  acta_entrega_insert = row_to_acta_entrega(row, CLIENTES, actas_procesadas)

  etiqueta_insert = row_to_etiqueta(row, etiqueta_index)
  if not etiqueta_insert.startswith("--"):
    etiqueta_index += 1

  activo_actaentrega_insert = row_to_activo_actaentrega(row, insert_index, etiqueta_index - 1)

  output_sql.append(activo_insert)
  output_sql.append(atributo_activo_insert)
  output_sql.append(observacion_insert)
  output_sql.append(movimiento_insert)
  output_sql.append(acta_entrega_insert)
  output_sql.append(etiqueta_insert)
  output_sql.append(activo_actaentrega_insert)

  insert_index += 1

# Acomodar secuencias de indices al final
reset_seq_sql = reset_sequences()
output_sql.append("\n/* ======= REINICIAR SECUENCIAS ======= */\n")
output_sql.append(reset_seq_sql)

# Agregar estadísticas al final
output_sql.append(f"\n/* ======= ESTADÍSTICAS ======= */")
output_sql.append(f"/* Total de activos procesados: {insert_index - 1} */")
output_sql.append(f"/* Total de actas únicas encontradas: {len(actas_procesadas)} */")
output_sql.append(f"/* Total de etiquetas generadas: {etiqueta_index - 1} */")
output_sql.append(f"/* Total de observaciones: {observacion_index - 1} */")
output_sql.append(f"/* Total de movimientos: {movimiento_index - 1} */")

with open(output_sql_inserts, "w", encoding="utf-8") as f:
    f.write("\n".join(output_sql))

print(f"Script completado. Archivo generado: {output_sql_inserts}")
print(f"Actas únicas procesadas: {len(actas_procesadas)}")
print(f"Activos procesados: {insert_index - 1}")

/tmp/ipykernel_2265/1939574963.py:8: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha_dt = pd.to_datetime(fecha_raw, dayfirst=True, errors='coerce')


Script completado. Archivo generado: output/AssetsInserts.sql
Actas únicas procesadas: 599
Activos procesados: 2014





---



---



---



---


---


---



---



---


# LO QUE SIGUE ES SOLO PARA PRUEBAS DE EJECUCIÓN


---


---



---




---



---



---



---



---



In [29]:
clients.head()

,id,idcrm,ltipocliente,lmoneda,lemailfacturar,lsector,name,type,identification,person_type,...,correo_facturacion_electronica,correo_reportar_pagos,sistemas_correo,sistemas_nombre,sistemas_telefono,tesoreria_correo,tesoreria_nombre,tesoreria_telefono,carteradetalladaID,codigo_cliente
0,365cb2e0-b13a-4c31-9ec0-462baad5ca19,1fd1783c-a65a-40bf-af25-8ed404727e80,Platino,COP,recepcionfacturas@e-tic.co,Telecomunicaciones,E-SOLUCIONES TIC S.A.S.,Customer,900823531,Company,...,,,,,,,,,None,178
1,73eaa659-e3a1-44d1-a35d-c91e3621a185,48387170-0dad-4dec-bac1-37d8bffff40b,Platino,COP,DCARVAJAL@DASERVICE.COM.CO,Construcción,D.A. SERVICE S.A.S.,Customer,901510063,Company,...,,,,,,,,,None,182
2,632588e1-f898-4e51-a6d9-31d83820b6f1,None,Platino,COP,asistente@eymconstructores.com.co,Construcción,E Y M CONSTRUCTORES S.A.S.,Customer,9002438225,Company,...,,,,,,,,,None,155
3,79c94e74-f582-4d98-ab98-45e16f4b2238,c0b43358-e840-4706-a302-db77ed93881e,Platino,COP,FINANCIERA@VIVIRENELPOBLADO.COM,Turismo,VIVIR EN EL POBLADO S.A.S.,Customer,901167844,Company,...,,,,,,,,,None,183
4,a48a44ec-2dad-4c87-8c1a-43c8ae9fabd3,e400b120-5e57-4cac-a285-b4c13ac48fd5,Platino,COP,CATALINA.OROZCO@OFFCORSS.COM,Producción Textil,C.I HERMECO S.A,Customer,890924167,Company,...,,,,,,,,,None,177


In [30]:
muestra = equipos.sample(n=10)

In [31]:
muestra.head()

,PLACA,TIPO,MARCA,MODELO,PANTALLA,PROCESADOR,GENERACION,MEMORIA,DISCO DURO,SERIAL FISICO,...,ACTA DE ENTREGA,CANON,SISTEMA OPERATIVO,MOVIMIENTO,COMPRADO A,FACTURA #,NUEVO / USADO,GARANTIA,FECHA COMPRA,FECHA FIN
1675,32662,PORTATIL,DELL,LATITUDE 5420,"14""",CORE I5,1135G7,2X8GB,M2 256GB,8FX21J3,...,A163-23,107642,WIN 11 PRO,NaN,EBAY,10-12798-22096,USADO,NaN,2025-03-07 00:00:00,2026-03-19 00:00:00
1043,32037,CPU,DELL,OPTIPLEX 3070 MFF,NaN,CORE I5,9500T,1X8GB,SSD SATA 256GB,6ND3P23,...,A227-111,63360,WIN 11 PRO,NaN,CAMBIO OFI,NaN,USADO,NaN,NaN,2026-05-19 00:00:00
1764,32734,PORTATIL,DELL,LATITUDE 5420,"14""",CORE I5,1145G7,1X16GB,M2 512GB,CT581J3,...,A192-30,123911,WIN 11 PRO,NaN,EBAY,14-12896-50808,USADO,NaN,2025-04-01 00:00:00,2025-09-25 00:00:00
1117,32087,PORTATIL,DELL,LATITUDE 5410,"14""",CORE I7,10610U,1X16GB,M2 512GB,C6QZ693,...,A177-51,NaN,WIN 11 PRO,NaN,EBAY,24-11598-27812,USADO,NaN,2024-05-21 00:00:00,2025-04-18 00:00:00
1404,32367,CPU,DELL,OPTIPLEX 7080 SFF,NaN,CORE I5,10500T,2X8GB,M2 512GB,30V7H63,...,A177-80,NaN,WIN 11 PRO,NaN,EBAY,16-12199-39761,USADO,NaN,2024-10-17 00:00:00,2025-11-06 00:00:00


In [4]:
df["ESTADO"].unique()

array(['Asignado', 'Stock', 'Pend. Dev. Cliente', 'Revisión',
       'Pend. X Repuesto', 'Reparación', 'Acondicionamiento'],
      dtype=object)

In [37]:
equipos.columns

Index(['PLACA', 'SUBTIPO', 'MARCA', 'MODELO', 'PANTALLA', 'PROCESADOR',
       'GENERACION', 'MEMORIA', 'DISCO DURO', 'SERIAL FISICO', 'OBSERVACIONES',
       'SERIAL BOARD', 'ESTADO', 'UBICACION', 'ASIGNADO A', 'ACTA DE ENTREGA',
       'CANON', 'SISTEMA OPERATIVO', 'MOVIMIENTO', 'PROVEEDOR', 'FACTURA #',
       'NUEVO / USADO', 'GARANTIA', 'FECHA COMPRA', 'FECHA FIN'],
      dtype='object')

In [34]:
equipos.dtypes

PLACA                object
TIPO                 object
MARCA                object
MODELO               object
PANTALLA             object
PROCESADOR           object
GENERACION           object
MEMORIA              object
DISCO DURO           object
SERIAL FISICO        object
OBSERVACIONES        object
SERIAL BOARD         object
ESTADO               object
UBICACION            object
ASIGNADO A           object
ACTA DE ENTREGA      object
CANON                object
SISTEMA OPERATIVO    object
MOVIMIENTO           object
COMPRADO A           object
FACTURA #            object
NUEVO / USADO        object
GARANTIA             object
FECHA COMPRA         object
FECHA FIN            object
dtype: object

In [7]:
df.dtypes

PLACA                 object
SUBTIPO               object
MARCA                 object
MODELO                object
PANTALLA              object
PROCESADOR            object
GENERACIÓN            object
MEMORIA               object
DISCO DURO            object
SERIAL FISICO         object
OBSERVACIONES         object
SERIAL BOARD          object
ESTADO                object
UBICACIÓN             object
ASIGNADO A            object
ACTA DE ENTREGA       object
Canon                float64
SISTEMA OPERATIVO     object
MOVIMIENTO            object
Comprado a            object
Factura #             object
Nuevo / Usado         object
Garantia              object
Fecha Compra          object
Fecha Fin            float64
dtype: object

In [36]:
cliente_codigo = safe_int(extraer_codigo_cliente("A-102"))
cliente_row = CLIENTES[CLIENTES['codigo_cliente'] == cliente_codigo]

print(cliente_row.index[0] + 1 if not cliente_row.empty else "NULL")

print(cliente_codigo)

NULL
None


In [38]:
df.columns

Index(['PLACA', 'SUBTIPO', 'MARCA', 'MODELO', 'PANTALLA', 'PROCESADOR',
       'GENERACIÓN', 'MEMORIA', 'DISCO DURO', 'SERIAL FISICO', 'OBSERVACIONES',
       'SERIAL BOARD', 'ESTADO', 'UBICACIÓN', 'ASIGNADO A', 'ACTA DE ENTREGA',
       'Canon', 'SISTEMA OPERATIVO', 'MOVIMIENTO', 'Comprado a', 'Factura #',
       'Nuevo / Usado', 'Garantia', 'Fecha Compra', 'Fecha Fin'],
      dtype='object')

In [39]:
equipos.columns

Index(['PLACA', 'SUBTIPO', 'MARCA', 'MODELO', 'PANTALLA', 'PROCESADOR',
       'GENERACION', 'MEMORIA', 'DISCO DURO', 'SERIAL FISICO', 'OBSERVACIONES',
       'SERIAL BOARD', 'ESTADO', 'UBICACION', 'ASIGNADO A', 'ACTA DE ENTREGA',
       'CANON', 'SISTEMA OPERATIVO', 'MOVIMIENTO', 'PROVEEDOR', 'FACTURA #',
       'NUEVO / USADO', 'GARANTIA', 'FECHA COMPRA', 'FECHA FIN'],
      dtype='object')

In [30]:
print(subestados_activos_insert)

INSERT INTO subestado_activo (subestado, estado_padre) VALUES
    ('ARRENDADO', 'ACTIVO'),
    ('ASIGNADO', 'ACTIVO'),
    ('PEND. DEV. CLIENTE', 'ACTIVO'),
    ('PRESTADO', 'ACTIVO'),
    ('REPARACION', 'ACTIVO'),
    ('STOCK', 'ACTIVO'),
    ('STOCK-PENDIENTE', 'ACTIVO'),
    ('GARANTIA', 'ACTIVO'),
    ('VENTA', 'OBSOLETO'),
    ('DONACION', 'OBSOLETO'),
    ('DAÑO', 'OBSOLETO'),
    ('BASURA ELECTRONICA', 'RETIRADO'),
    ('VENDIDO', 'RETIRADO'),
    ('ROBADO', 'RETIRADO'),
    ('PERDIDO', 'RETIRADO'),
    ('EXTRAER REPUESTOS', 'RETIRADO');

